# Main notebook for data cleaning

In [1]:
# standard library imports
import os

# third party imports
import numpy as np
import pandas as pd

# local imports


data_dir = os.path.join(os.path.dirname("__file__"), "..", "..", "data")
raw_data_dir = os.path.join(data_dir, "raw")
clean_data_dir = os.path.join(data_dir, "clean")

## MMA Decisions

- `events`, `judges`, and `media_scores` are already cleaned

In [10]:
dupe_fighter_map = {
    4934: 6373,
    6748: 6394,
}

In [15]:
mmadecisions_fighters = pd.read_csv(os.path.join(raw_data_dir, "MMA Decisions", "fighters.csv"), parse_dates=["date_of_birth"])

# Duplicate fighters
dupes = [4934, 6748]
mmadecisions_fighters = mmadecisions_fighters[~mmadecisions_fighters["id"].isin(dupes)]

def convert_height(height_str):
    if pd.isna(height_str):
        return np.nan
    
    height_str_split = height_str.split("'")
    feet = height_str_split[0]
    inches = height_str_split[1]
    feet = int(feet)
    inches = float(inches.replace('"', ''))

    return feet * 12 + inches

mmadecisions_fighters["height"] = mmadecisions_fighters["height"].apply(convert_height)
mmadecisions_fighters = mmadecisions_fighters.rename(columns={"height": "height_inches"})

mmadecisions_fighters.to_csv(os.path.join(clean_data_dir, "MMA Decisions", "fighters.csv"), index=False)

In [33]:
mmadecisions_bouts = pd.read_csv(os.path.join(raw_data_dir, "MMA Decisions", "bouts.csv"))

# replace duplicate fighter IDs
mmadecisions_bouts["fighter_1_id"] = mmadecisions_bouts["fighter_1_id"].replace(dupe_fighter_map)
mmadecisions_bouts["fighter_2_id"] = mmadecisions_bouts["fighter_2_id"].replace(dupe_fighter_map)

mmadecisions_bouts.to_csv(os.path.join(clean_data_dir, "MMA Decisions", "bouts.csv"), index=False)

In [38]:
mmadecisions_deductions = pd.read_csv(os.path.join(raw_data_dir, "MMA Decisions", "deductions.csv"))
mmadecisions_deductions["fighter_id"] = mmadecisions_deductions["fighter_id"].replace(dupe_fighter_map)
mmadecisions_deductions.to_csv(os.path.join(clean_data_dir, "MMA Decisions", "deductions.csv"), index=False)

In [42]:
mmadecisions_judge_scores = pd.read_csv(os.path.join(raw_data_dir, "MMA Decisions", "judge_scores.csv"))
mmadecisions_judge_scores["judge_id"] = mmadecisions_judge_scores["judge_id"].astype("Int64")
mmadecisions_judge_scores["fighter_1_score"] = mmadecisions_judge_scores["fighter_1_score"].astype("Int64")
mmadecisions_judge_scores["fighter_2_score"] = mmadecisions_judge_scores["fighter_2_score"].astype("Int64")

mmadecisions_judge_scores.to_csv(os.path.join(clean_data_dir, "MMA Decisions", "judge_scores.csv"), index=False)

## Bet MMA

- `missed_weights`, `late_replacements`, and `bouts` are already cleaned

In [102]:
betmma_fighters = pd.read_csv(os.path.join(raw_data_dir, "Bet MMA", "fighters.csv"))
betmma_fighters["sherdog_id"] = betmma_fighters["sherdog_id"].astype("Int64")
betmma_fighters = betmma_fighters.rename(columns={"reach": "reach_inches"})

def convert_height(height_str):
    if pd.isna(height_str):
        return np.nan
    
    height_str = height_str.replace('"', "")
    feet, inches = map(float, height_str.split("'"))

    return feet * 12 + inches

betmma_fighters["height_inches"] = betmma_fighters["height"].apply(convert_height)
betmma_fighters.loc[betmma_fighters["nationality"] == "-", "nationality"] = np.nan
betmma_fighters.loc[betmma_fighters["nationality"].str.endswith("d'Ivoire", na=False), "nationality"] = "Côte d'Ivoire"
betmma_fighters.loc[betmma_fighters["nationality"] == "Trinidad &amp; Tobago", "nationality"] = "Trinidad and Tobago"
betmma_fighters.loc[betmma_fighters["nationality"] == "Bosnia &amp; Herzegovina", "nationality"] = "Bosnia and Herzegovina"

betmma_fighters = betmma_fighters[
    ['id',
 'ufcstats_id',
 'sherdog_id',
 'name',
 'height_inches',
 'reach_inches',
 'stance',
 'nationality']
]

betmma_fighters.loc[betmma_fighters["reach_inches"] > 100, "reach_inches"] = np.nan

betmma_fighters.to_csv(os.path.join(clean_data_dir, "Bet MMA", "fighters.csv"), index=False)

In [124]:
betmma_fighter_histories = pd.read_csv(os.path.join(raw_data_dir, "Bet MMA", "fighter_histories.csv"))
betmma_fighter_histories["outcome"] = betmma_fighter_histories["outcome"].map({"Won": "W", "Lost": "L", "Draw": "D"})
betmma_fighter_histories.loc[betmma_fighter_histories["end_round"] == 0, "end_round"] = np.nan
betmma_fighter_histories["end_round"] = betmma_fighter_histories["end_round"].astype("Int64")
betmma_fighter_histories.loc[betmma_fighter_histories["end_round_time"] == "0:00", "end_round_time"] = np.nan

def convert_time(time_str):
    if pd.isna(time_str):
        return np.nan
    
    minutes, seconds = map(int, time_str.split(":"))
    return minutes * 60 + seconds

betmma_fighter_histories["end_round_time_seconds"] = betmma_fighter_histories["end_round_time"].apply(convert_time)
betmma_fighter_histories["end_round_time_seconds"] = betmma_fighter_histories["end_round_time_seconds"].astype("Int64")

def total_seconds(row):
    if pd.isna(row["end_round_time_seconds"]) or pd.isna(row["end_round"]):
        return np.nan
    return (row["end_round"] - 1) * 300 + row["end_round_time_seconds"]

betmma_fighter_histories["total_time_seconds"] = betmma_fighter_histories.apply(total_seconds, axis=1)
betmma_fighter_histories["total_time_seconds"] = betmma_fighter_histories["total_time_seconds"].astype("Int64")

betmma_fighter_histories["odds"] = betmma_fighter_histories["odds"].astype("Int64")

betmma_fighter_histories = betmma_fighter_histories[
    ['fighter_id',
 'order',
 'bout_id',
 'opponent_id',
 'outcome',
 'outcome_method',
 'end_round',
 'end_round_time_seconds',
 'total_time_seconds',
 'odds']
]

betmma_fighter_histories.to_csv(os.path.join(clean_data_dir, "Bet MMA", "fighter_histories.csv"), index=False)

In [131]:
betmma_events = pd.read_csv(os.path.join(raw_data_dir, "Bet MMA", "events.csv"))

betmma_events['temp_event_order'] = (
    betmma_events['is_ufc_event']
    .eq(1)  # Check where the column equals 1
    .cumsum()  # Cumulative sum to count occurrences
    .where(betmma_events['is_ufc_event'] == 1, np.nan)  # Set to NaN where the trigger column is 0
)
betmma_events["event_order"] = 233 + betmma_events["temp_event_order"]
betmma_events["event_order"] = betmma_events["event_order"].astype("Int64")

betmma_events = betmma_events[
    ['id',
 'name',
 'date',
 'location',
 'is_ufc_event',
 'event_order']
]

betmma_events.to_csv(os.path.join(clean_data_dir, "Bet MMA", "events.csv"), index=False)

## ESPN

- `teams` and `venues` are already cleaned

In [18]:
espn_fighters = pd.read_csv(os.path.join(raw_data_dir, "ESPN", "fighters.csv"))
espn_fighters["team_id"] = espn_fighters["team_id"].astype("Int64")
espn_fighters["date_of_birth"] = pd.to_datetime(espn_fighters["date_of_birth"], format="%d/%m/%Y")
espn_fighters.loc[espn_fighters["stance"] == "--", "stance"] = np.nan
espn_fighters["reach_inches"] = espn_fighters["reach"].str.replace('"', "").astype("float")

def convert_height(height_str):
    if pd.isna(height_str):
        return np.nan
    
    height_str = height_str.replace("'", "").replace('"', "")
    feet, inches = map(int, height_str.split())

    return feet * 12 + inches

espn_fighters["height_inches"] = espn_fighters["height"].apply(convert_height)
espn_fighters["height_inches"] = espn_fighters["height_inches"].astype("Int64")

espn_fighters = espn_fighters[
    ['id',
 'name',
 'nickname',
 'date_of_birth',
 'reach_inches',
 'height_inches',
 'stance',
 'team_id',
 'nationality',
 'fighting_style',]
]

espn_fighters.to_csv(os.path.join(clean_data_dir, "ESPN", "fighters.csv"), index=False)

In [26]:
espn_events = pd.read_csv(os.path.join(raw_data_dir, "ESPN", "events.csv"))
espn_events["venue_id"] = espn_events["venue_id"].astype("Int64")
espn_events = espn_events.rename(columns={"date": "date_TEMP"})

espn_events["date"] = pd.to_datetime(espn_events["date_TEMP"]).dt.date
espn_events["hour_utc"] = pd.to_datetime(espn_events["date_TEMP"]).dt.hour

espn_events = espn_events[["id", "name", "date", "hour_utc", "venue_id", "event_order"]]

espn_events.to_csv(os.path.join(clean_data_dir, "ESPN", "events.csv"), index=False)

In [29]:
espn_bouts = pd.read_csv(os.path.join(raw_data_dir, "ESPN", "bouts.csv"))
espn_bouts["winner_id"] = espn_bouts["winner_id"].astype("Int64")
espn_bouts.to_csv(os.path.join(clean_data_dir, "ESPN", "bouts.csv"), index=False)

In [45]:
espn_fighter_histories = pd.read_csv(os.path.join(raw_data_dir, "ESPN", "fighter_histories.csv"))
espn_fighter_histories["opponent_id"] = espn_fighter_histories["opponent_id"].astype("Int64")

espn_fighter_histories = espn_fighter_histories.rename(columns={"date": "date_TEMP"})
espn_fighter_histories["date"] = pd.to_datetime(espn_fighter_histories["date_TEMP"]).dt.date
espn_fighter_histories["hour_utc"] = pd.to_datetime(espn_fighter_histories["date_TEMP"]).dt.hour

espn_fighter_histories["end_round"] = espn_fighter_histories["end_round"].astype("Int64")
espn_fighter_histories.loc[espn_fighter_histories["end_round"] == 0, "end_round"] = np.nan

def convert_time(time_str):
    if pd.isna(time_str) or time_str == "-":
        return np.nan
    
    minutes, seconds = map(int, time_str.split(":"))
    return minutes * 60 + seconds

espn_fighter_histories["end_round_time_seconds"] = espn_fighter_histories["end_round_time"].apply(convert_time)
espn_fighter_histories["end_round_time_seconds"] = espn_fighter_histories["end_round_time_seconds"].astype("Int64")

def calculate_total_time_seconds(row):
    if pd.isna(row["end_round"]) or pd.isna(row["end_round_time_seconds"]):
        return np.nan
    
    return (row["end_round"] - 1) * 300 + row["end_round_time_seconds"]

espn_fighter_histories["total_time_seconds"] = espn_fighter_histories.apply(calculate_total_time_seconds, axis=1)
espn_fighter_histories["total_time_seconds"] = espn_fighter_histories["total_time_seconds"].astype("Int64")

espn_fighter_histories = espn_fighter_histories[
    ['fighter_id',
 'order',
 'bout_id',
 'event_id',
 'event_name',
 'date',
 'hour_utc',
 'opponent_id',
 'outcome',
 'outcome_method',
 'end_round',
 'end_round_time_seconds',
  'total_time_seconds',
 'is_title_bout']
]

espn_fighter_histories.to_csv(os.path.join(clean_data_dir, "ESPN", "fighter_histories.csv"), index=False)

In [72]:
espn_bout_stats = pd.read_csv(os.path.join(raw_data_dir, "ESPN", "fighter_bout_statistics.csv"))

stat_cols = [x for x in espn_bout_stats.columns if x not in {'fighter_id','order','bout_id','event_id',}]
espn_bout_stats[stat_cols] = espn_bout_stats[stat_cols].astype("Int64")

espn_bout_stats = espn_bout_stats.drop(columns=["order", "event_id"])
espn_bout_stats = espn_bout_stats[["bout_id", "fighter_id"] + stat_cols]
espn_bout_stats = espn_bout_stats.sort_values(by=["bout_id", "fighter_id"]).reset_index(drop=True)

espn_bout_stats.to_csv(os.path.join(clean_data_dir, "ESPN", "bout_stats.csv"), index=False)

## UFC Stats

- `fighter_histories` is already cleaned

In [ ]:
ufcstats_fighters = pd.read_csv(os.path.join(raw_data_dir, "UFC Stats", "fighters.csv"), parse_dates=["date_of_birth"])
# ufcstats_fighters

# Convert height and reach to integers, defaulted to float because of NaNs
ufcstats_fighters[["height_inches", "reach_inches"]] = ufcstats_fighters[["height_inches", "reach_inches"]].astype("Int64")
ufcstats_fighters.to_csv(os.path.join(clean_data_dir, "UFC Stats", "fighters.csv"), index=False)

In [20]:
ufcstats_events = pd.read_csv(os.path.join(raw_data_dir, "UFC Stats", "events.csv"), parse_dates=["date"])
# ufcstats_events

ufcstats_events["event_order"] = ufcstats_events["event_order"].astype("Int64")
ufcstats_events.to_csv(os.path.join(clean_data_dir, "UFC Stats", "events.csv"), index=False)

In [23]:
ufcstats_round_stats = pd.read_csv(os.path.join(raw_data_dir, "UFC Stats", "round_stats.csv"))
# ufcstats_round_stats.dtypes

ufcstats_round_stats["control_time_seconds"] = ufcstats_round_stats["control_time_seconds"].astype("Int64")
ufcstats_round_stats.to_csv(os.path.join(clean_data_dir, "UFC Stats", "round_stats.csv"), index=False)

In [45]:
bout_ids_to_flip = [
    "ca93e3f69fa3d725",
    "b4d624bdc27dff83",
    "aefca2869c87eb11",
    "0ea087a71863184d",
    "b091e021e61f1950",
    "5e52b0bf9719f0ae",
    "840863604b38a33f",
    "adddc6e46da5ca19",
    "20628fd4e19a97e4",
    "1a21263dc5d866b6",
    "504b540805598fa5",
    "be72958d9715757d",
    "19f615a7a5cfd304",
    "c4fa93a4f37a6ca7",
    "c99370e3e54bd5fd",
    "d7f9a09021a9a13c",
    "24e14c4824144c64",
]
column_swap_map = {
    "red_fighter_id": "blue_fighter_id",
    "blue_fighter_id": "red_fighter_id",
    "red_outcome": "blue_outcome",
    "blue_outcome": "red_outcome",
}

ufcstats_bouts = pd.read_csv(os.path.join(raw_data_dir, "UFC Stats", "bouts.csv"))

# flip fighter ids and outcomes for specific bouts
ufcstats_bouts.update(
    ufcstats_bouts[ufcstats_bouts["id"].isin(bout_ids_to_flip)]
    .rename(columns=column_swap_map)
)

ufcstats_bouts.to_csv(os.path.join(clean_data_dir, "UFC Stats", "bouts.csv"), index=False)

## Wikipedia

- `venues` is already cleaned

In [25]:
wikipedia_events = pd.read_csv(os.path.join(raw_data_dir, "Wikipedia", "events.csv"), parse_dates=["date"])
wikipedia_events["attendance"] = wikipedia_events["attendance"].astype("Int64")
wikipedia_events.to_csv(os.path.join(clean_data_dir, "Wikipedia", "events.csv"), index=False)

## Sherdog

In [27]:
sherdog_fighters = pd.read_csv(os.path.join(raw_data_dir, "Sherdog", "fighters.csv"), parse_dates=["date_of_birth", "pro_debut_date"])
sherdog_fighters["height_inches"] = sherdog_fighters["height_inches"].astype("Int64")
sherdog_fighters.to_csv(os.path.join(clean_data_dir, "Sherdog", "fighters.csv"), index=False)

In [38]:
sherdog_events = pd.read_csv(os.path.join(raw_data_dir, "Sherdog", "events.csv"), parse_dates=["date"])
sherdog_events["name"] = sherdog_events["name"].str.replace("\n", " ", regex=False).replace("  ", " ", regex=False).str.strip()
sherdog_events["location"] = sherdog_events["location"].str.replace("\r", ",", regex=False).replace("\n", " ", regex=False).replace("  ", " ", regex=False).str.strip()
sherdog_events["country"] = sherdog_events["country"].str.split('\r\n').str[0].str.strip()
sherdog_events["event_order"] = sherdog_events["event_order"].astype("Int64")
sherdog_events.loc[sherdog_events["date"] == "2030-01-11", "date"] = pd.to_datetime("2022-11-30")
sherdog_events = sherdog_events.sort_values(by=["date", "event_order"]).reset_index(drop=True)

# print(sherdog_events.loc[sherdog_events["id"] == 97481, "location"].values)

sherdog_events.to_csv(os.path.join(clean_data_dir, "Sherdog", "events.csv"), index=False)

In [145]:
sherdog_fighter_histories = pd.read_csv(os.path.join(raw_data_dir, "Sherdog", "fighter_histories.csv"))
sherdog_fighter_histories["opponent_id"] = sherdog_fighter_histories["opponent_id"].astype("Int64")
sherdog_fighter_histories["outcome"] = sherdog_fighter_histories["outcome"].map({"win": "W", "loss": "L", "draw": "D", "nc": "NC"})

sherdog_fighter_histories["outcome_method_broad"] = sherdog_fighter_histories["outcome_method"].str.split(r'\(|\)', expand=True).iloc[:, 0]
sherdog_fighter_histories["outcome_method_broad"] = sherdog_fighter_histories["outcome_method_broad"].str.strip()

# stupid edge cases
sherdog_fighter_histories.loc[sherdog_fighter_histories["outcome_method_broad"] == "", "outcome_method_broad"] = np.nan

dqs = ["Disqualification", "Disqualifcation", "Desqualification", "DG", "DQ"]
sherdog_fighter_histories.loc[sherdog_fighter_histories["outcome_method_broad"].isin(dqs), "outcome_method_broad"] = "Disqualification"

ko_tkos = ["ТКО", "Tko", "TKP", "ΤΚΟ", "Corner Stoppage", "Corner's towel", "Doctor's Stoppage", "Injury", "Retirement",
           "K.O", "K.O.", "KO", "Ko", "Knockout", "ko", "KO/TKO", "KO/TKO PUNCHES", "TKO Punches", "TKO", "RETIREMENT"]
sherdog_fighter_histories.loc[sherdog_fighter_histories["outcome_method_broad"].isin(ko_tkos), "outcome_method_broad"] = "KO/TKO"

draws = ["DRAW", "Draw Unanimous", "Drew", "Majority Draw", "Technical Draw"]
sherdog_fighter_histories.loc[sherdog_fighter_histories["outcome_method_broad"].isin(draws), "outcome_method_broad"] = "Draw"

decisions = ["Decision", "Decision Unanimous", "Decision unanimous", "Decisions", "Decison", "Decisão Unanime", "Desicion",
             "Desision", "Majority Decision", "Points", "Split Decision", "Split Division", "Split decision", "Technical Decision",
             "Unaminous Decision", "Unanimous", "Unanimous Decision", "Unanimous decision", "de"]
sherdog_fighter_histories.loc[sherdog_fighter_histories["outcome_method_broad"].isin(decisions), "outcome_method_broad"] = "Decision"

subs = ["Frontal Anaconda Choke", "Guillotine Choke", "Guillotine choke", "Kraken Choke", "Rear Naked Choke", "SUBMISSION",
        "Submision", "Submisison", "Submissio", "Submission", "Submissions", "Submisson", "Submssion", "Tapout",
        "Techinal Submission", "Techincal Submission", "Technial Submission", "Technical Submission", "Triangle Choke",
        "Verbal Submission", "su", "submison", "submission", "ubmission"]
sherdog_fighter_histories.loc[sherdog_fighter_histories["outcome_method_broad"].isin(subs), "outcome_method_broad"] = "Submission"

ncs = ["N/C", "NC", "ND", "No Conest", "No Contest", "No Contest - Clements Failed Drug Test", 
       "No Contest - Collard Failed Drug Test", "No Contest - Overturned by Commission", 
       "No Contest - Overturned by NSAC", "No Contest - Overturned by WTKA", "No Contest - Result overturned by FIGMMA",
       "No Contest - Strikes After The Fight", "No Decision", "No Decision - Overturned by CSAC",
       "No Decision - Overturned by FMMAF", "No Decision - Zappitella Failed Drug Test", "Overtuned",
       "Overturned by Promoter"]
sherdog_fighter_histories.loc[sherdog_fighter_histories["outcome_method_broad"].isin(ncs), "outcome_method_broad"] = "No Contest"

# end round stuff
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round"] == 0, "end_round"] = np.nan
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round"] == 12, "end_round"] = 1
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round"] == 22, "end_round"] = 2
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round"] == 30, "end_round"] = 3
sherdog_fighter_histories["end_round"] = sherdog_fighter_histories["end_round"].astype("Int64")

# end round time
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace(";", ":", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace('"', ":", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace(".", ":", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace(",", ":", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace("?", ":", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace("L", ":", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace("_", ":", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace(" min", "", regex=False)
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "M/A", "end_round_time"] = np.nan
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "SUBMISSION", "end_round_time"] = np.nan
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"].isin(["1", "2", "3", "5"]), "end_round_time"] += ":00"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "05", "end_round_time"] = "5:00"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "38", "end_round_time"] = "0:38"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "51", "end_round_time"] = "0:51"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "021", "end_round_time"] = "0:21"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "215", "end_round_time"] = "2:15"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "311", "end_round_time"] = "3:11"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "201641", "end_round_time"] = "5:00"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "383518", "end_round_time"] = np.nan
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "429661", "end_round_time"] = np.nan
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "2:30::", "end_round_time"] = "2:30"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "2: 5", "end_round_time"] = "2:05"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "2:26:", "end_round_time"] = "2:26"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "01:15:00", "end_round_time"] = "1:15"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "00:03:50", "end_round_time"] = "3:50"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "05:00:00", "end_round_time"] = "5:00"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "00:02:48", "end_round_time"] = "2:48"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "00:05:00", "end_round_time"] = "5:00"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "04:17:00", "end_round_time"] = "4:17"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "00:01:39", "end_round_time"] = "1:39"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "4:45:00", "end_round_time"] = "4:45"
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time"] == "1:12:00 AM", "end_round_time"] = "1:12"
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace("!", "1", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace("$", "4", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace("&", "7", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace("q", "", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace("d", "", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace("'", "", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace("::", ":", regex=False)
sherdog_fighter_histories["end_round_time"] = sherdog_fighter_histories["end_round_time"].str.replace(" :", ":", regex=False)

# end round time seconds
def convert_to_seconds(time_str):
    if pd.isna(time_str):
        return np.nan
    minutes, seconds = time_str.split(":")
    if minutes == "":
        minutes = 0
    
    minutes = int(minutes)
    seconds = int(seconds)

    return minutes * 60 + seconds

sherdog_fighter_histories["end_round_time_seconds"] = sherdog_fighter_histories["end_round_time"].apply(convert_to_seconds)
sherdog_fighter_histories.loc[sherdog_fighter_histories["end_round_time_seconds"] == 0, "end_round_time_seconds"] = np.nan
sherdog_fighter_histories["end_round_time_seconds"] = sherdog_fighter_histories["end_round_time_seconds"].astype("Int64")

# total time seconds
def calculate_total_time_seconds(row):
    if pd.isna(row["end_round"]) or pd.isna(row["end_round_time_seconds"]):
        return np.nan

    return (row["end_round"] - 1) * 300 + row["end_round_time_seconds"]

sherdog_fighter_histories["total_time_seconds"] = sherdog_fighter_histories.apply(calculate_total_time_seconds, axis=1)
sherdog_fighter_histories["total_time_seconds"] = sherdog_fighter_histories["total_time_seconds"].astype("Int64")

sherdog_fighter_histories.loc[sherdog_fighter_histories["event_id"] == 95098, "date"] = pd.to_datetime("2022-11-30")

sherdog_fighter_histories = sherdog_fighter_histories.sort_values(by=["fighter_id", "date", "order"]).reset_index(drop=True)
sherdog_fighter_histories["order"] = sherdog_fighter_histories.groupby("fighter_id").cumcount() + 1

sherdog_fighter_histories = sherdog_fighter_histories[
    ['fighter_id',
    'order',
    'event_id',
    'date',
    'opponent_id',
    'outcome',
    'outcome_method',
    'outcome_method_broad',
    'end_round',
    'end_round_time_seconds',
    'total_time_seconds']
]

sherdog_fighter_histories.to_csv(os.path.join(clean_data_dir, "Sherdog", "fighter_histories.csv"), index=False)

In [55]:
sherdog_bouts = pd.read_csv(os.path.join(raw_data_dir, "Sherdog", "bouts.csv"))
sherdog_bouts["fighter_1_id"] = sherdog_bouts["fighter_1_id"].astype("Int64")
sherdog_bouts["fighter_2_id"] = sherdog_bouts["fighter_2_id"].astype("Int64")

outcome_map = {
    "win": "W",
    "loss": "L",
    "draw": "D",
    "nc": "NC",
    "yet to come": np.nan
}
sherdog_bouts["fighter_1_outcome"] = sherdog_bouts["fighter_1_outcome"].map(outcome_map)
sherdog_bouts["fighter_2_outcome"] = sherdog_bouts["fighter_2_outcome"].map(outcome_map)

sherdog_bouts = sherdog_bouts.rename(columns={"weight_class": "weight_class_TEMP"})
sherdog_bouts["weight_class"] = sherdog_bouts["weight_class_TEMP"].apply(lambda x: np.nan if pd.isna(x) else "Catchweight" if "Catchweight" in x else np.nan if "lb" in x else x)
sherdog_bouts.loc[sherdog_bouts["weight_class"] == "Pound for Pound", "weight_class"] = np.nan

def extract_weight_lbs(weight_class_str):
    weight_map = {
        "Atomweight": 105,
        "Strawweight": 115,
        "Flyweight": 125,
        "Bantamweight": 135,
        "Featherweight": 145,
        "Lightweight": 155,
        "Welterweight": 170,
        "Middleweight": 185,
        "Light Heavyweight": 205,
        "Heavyweight": 265,
    }

    if pd.isna(weight_class_str):
        return np.nan
    elif "lb" in weight_class_str:
        return int(weight_class_str.replace("lb", "").replace("Catchweight", ""))
    
    return weight_map.get(weight_class_str, np.nan)

sherdog_bouts["weight_class_lbs"] = sherdog_bouts["weight_class_TEMP"].apply(extract_weight_lbs)
sherdog_bouts["weight_class_lbs"] = sherdog_bouts["weight_class_lbs"].astype("Int64")


sherdog_bouts["outcome_method_broad"] = sherdog_bouts["outcome_method"].str.split(r'\(|\)', expand=True).iloc[:, 0]
sherdog_bouts["outcome_method_broad"] = sherdog_bouts["outcome_method_broad"].str.strip()

# stupid edge cases
sherdog_bouts.loc[sherdog_bouts["outcome_method_broad"] == "", "outcome_method_broad"] = np.nan

dqs = ["Disqualification", "Disqualifcation", "Desqualification", "DG", "DQ"]
sherdog_bouts.loc[sherdog_bouts["outcome_method_broad"].isin(dqs), "outcome_method_broad"] = "Disqualification"

ko_tkos = ["ТКО", "Tko", "TKP", "ΤΚΟ", "Corner Stoppage", "Corner's towel", "Doctor's Stoppage", "Injury", "Retirement",
           "K.O", "K.O.", "KO", "Ko", "Knockout", "ko", "KO/TKO", "KO/TKO PUNCHES", "TKO Punches", "TKO", "RETIREMENT",
           "Dcotor Stoppade", "Nocaute"]
sherdog_bouts.loc[sherdog_bouts["outcome_method_broad"].isin(ko_tkos), "outcome_method_broad"] = "KO/TKO"

draws = ["DRAW", "Draw Unanimous", "Drew", "Majority Draw", "Technical Draw"]
sherdog_bouts.loc[sherdog_bouts["outcome_method_broad"].isin(draws), "outcome_method_broad"] = "Draw"

decisions = ["Decision", "Decision Unanimous", "Decision unanimous", "Decisions", "Decison", "Decisão Unanime", "Desicion",
             "Desision", "Majority Decision", "Points", "Split Decision", "Split Division", "Split decision", "Technical Decision",
             "Unaminous Decision", "Unanimous", "Unanimous Decision", "Unanimous decision", "de", "Decisionn", "Deicision"]
sherdog_bouts.loc[sherdog_bouts["outcome_method_broad"].isin(decisions), "outcome_method_broad"] = "Decision"

subs = ["Frontal Anaconda Choke", "Guillotine Choke", "Guillotine choke", "Kraken Choke", "Rear Naked Choke", "SUBMISSION",
        "Submision", "Submisison", "Submissio", "Submission", "Submissions", "Submisson", "Submssion", "Tapout",
        "Techinal Submission", "Techincal Submission", "Technial Submission", "Technical Submission", "Triangle Choke",
        "Verbal Submission", "su", "submison", "submission", "ubmission", "Bulldog Choke", "RNC", "Submissiom",
        "Submission-Guillotine Choke", "Submissoin", "sub"]
sherdog_bouts.loc[sherdog_bouts["outcome_method_broad"].isin(subs), "outcome_method_broad"] = "Submission"

ncs = ["N/C", "NC", "ND", "No Conest", "No Contest", "No Contest - Clements Failed Drug Test", 
       "No Contest - Collard Failed Drug Test", "No Contest - Overturned by Commission", 
       "No Contest - Overturned by NSAC", "No Contest - Overturned by WTKA", "No Contest - Result overturned by FIGMMA",
       "No Contest - Strikes After The Fight", "No Decision", "No Decision - Overturned by CSAC",
       "No Decision - Overturned by FMMAF", "No Decision - Zappitella Failed Drug Test", "Overtuned",
       "Overturned by Promoter", "Illegal Punch", "Unintentional Eye Poke"]
sherdog_bouts.loc[sherdog_bouts["outcome_method_broad"].isin(ncs), "outcome_method_broad"] = "No Contest"

# end round stuff
sherdog_bouts.loc[sherdog_bouts["end_round"] == 0, "end_round"] = np.nan
sherdog_bouts.loc[sherdog_bouts["end_round"] == 12, "end_round"] = 1
sherdog_bouts.loc[sherdog_bouts["end_round"] == 22, "end_round"] = 2
sherdog_bouts.loc[sherdog_bouts["end_round"] == 30, "end_round"] = 3
sherdog_bouts.loc[sherdog_bouts["end_round"] == 31, "end_round"] = 1
sherdog_bouts.loc[sherdog_bouts["end_round"] == 4355842, "end_round"] = 3
sherdog_bouts["end_round"] = sherdog_bouts["end_round"].astype("Int64")

# end round time
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace(";", ":", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace('"', ":", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace(" :", ":", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace(")", "0", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace("!", "1", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace("$", "4", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace("&", "7", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace(".:", ":", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace(".", ":", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace("_", ":", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace("?", ":", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace("L", ":", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace("d", "", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace("q", "", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace("r", "", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace("s", "", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace("'", "", regex=False)
sherdog_bouts["end_round_time"] = sherdog_bouts["end_round_time"].str.replace(",", ":", regex=False)

nans = [":evon Shab", "Aabek To", "Anew He", "BMISSION:", "Joeph Pe", "KO/TKO: PU", "KO/TKO: ST", "M/A",
        "Nick Kaaze", "SUBMISSION"]
sherdog_bouts.loc[sherdog_bouts["end_round_time"].isin(nans), "end_round_time"] = np.nan

sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "2 m an 14", "end_round_time"] = "2:14"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "2 min 11e", "end_round_time"] = "2:11"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "2:50 min", "end_round_time"] = "2:50"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "2: 5", "end_round_time"] = "2:05"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "2:4u", "end_round_time"] = "2:40"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "KO/TKO 2:1", "end_round_time"] = "2:10"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "KO/TKO 2:5", "end_round_time"] = "2:50"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "KO/TKO 3:0", "end_round_time"] = "3:00"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "1:12:00 AM", "end_round_time"] = "1:12"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "4:40:00 AM", "end_round_time"] = "4:40"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "00:01:39", "end_round_time"] = "1:39"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "00:02:48", "end_round_time"] = "2:48"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "00:03:02", "end_round_time"] = "3:02"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "00:03:50", "end_round_time"] = "3:50"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "00:05:00", "end_round_time"] = "5:00"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "01:15:00", "end_round_time"] = "1:15"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "04:17:00", "end_round_time"] = "4:17"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "05:00:00", "end_round_time"] = "5:00"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "4:45:00", "end_round_time"] = "4:45"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "0::32", "end_round_time"] = "0:32"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "1::45", "end_round_time"] = "1:45"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "2:26:", "end_round_time"] = "2:26"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "2:30::", "end_round_time"] = "2:30"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "2::58", "end_round_time"] = "2:58"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "05", "end_round_time"] = "5:00"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "215", "end_round_time"] = "2:15"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "021", "end_round_time"] = "0:21"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "241", "end_round_time"] = "2:41"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "250", "end_round_time"] = "2:50"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "311", "end_round_time"] = "3:11"
sherdog_bouts.loc[sherdog_bouts["end_round_time"] == "135", "end_round_time"] = "1:35"

nans2 = ["1", '110087',
 '135',
 '2',
 '201641',
 '23',
 '3',
 '308073',
 '38',
 '383518',
 '429661',
 '431507',
 '467551',
 '5',
 '51']
sherdog_bouts.loc[sherdog_bouts["end_round_time"].isin(nans2), "end_round_time"] = np.nan

# end round time seconds
def convert_to_seconds(time_str):
    if pd.isna(time_str):
        return np.nan
    minutes, seconds = time_str.split(":")
    if minutes == "":
        minutes = 0
    
    minutes = int(minutes)
    seconds = int(seconds)

    return minutes * 60 + seconds

sherdog_bouts["end_round_time_seconds"] = sherdog_bouts["end_round_time"].apply(convert_to_seconds)
sherdog_bouts.loc[sherdog_bouts["end_round_time_seconds"] == 0, "end_round_time_seconds"] = np.nan
sherdog_bouts["end_round_time_seconds"] = sherdog_bouts["end_round_time_seconds"].astype("Int64")

# total time seconds
def calculate_total_time_seconds(row):
    if pd.isna(row["end_round"]) or pd.isna(row["end_round_time_seconds"]):
        return np.nan

    return (row["end_round"] - 1) * 300 + row["end_round_time_seconds"]

sherdog_bouts["total_time_seconds"] = sherdog_bouts.apply(calculate_total_time_seconds, axis=1)
sherdog_bouts["total_time_seconds"] = sherdog_bouts["total_time_seconds"].astype("Int64")

sherdog_bouts = sherdog_bouts[
    ['event_id',
 'bout_order',
 'fighter_1_id',
 'fighter_2_id',
 'fighter_1_outcome',
 'fighter_2_outcome',
 'is_title_bout',
 'weight_class',
 'weight_class_lbs',
 'outcome_method',
 'outcome_method_broad',
 'end_round',
 'end_round_time_seconds',
 'total_time_seconds']
]

sherdog_bouts.to_csv(os.path.join(clean_data_dir, "Sherdog", "bouts.csv"), index=False)